In [ ]:
import diffusers
from diffusers import UNet2DConditionModel
from PIL import Image
import sys
from transformers import CLIPVisionModel, CLIPImageProcessor
import torch

sys.path.append("/home/aihao/workspace/StableDiffusionReferenceOnly/src")
from stable_diffusion_reference_only.pipelines.stable_diffusion_reference_only_pipeline import (
    StableDiffusionReferenceOnlyPipeline,
)
from stable_diffusion_reference_only.models.unet_2d_dobule_condition import (
    UNet2DDobuleConditionModel,
)
from diffusers.schedulers import DDPMScheduler
import json

In [ ]:
unet_config_path = "/home/aihao/workspace/StableDiffusionReferenceOnly/src/stable_diffusion_reference_only/models/unet-2-1.json"
pretrained_unet_path = "stabilityai/stable-diffusion-2-1"
pretrained_image_encoder_path = "openai/clip-vit-large-patch14"

# unet_config_path = "/home/aihao/workspace/StableDiffusionReferenceOnly/src/stable_diffusion_reference_only/models/unet_xl-base-1.0.json"
# pretrained_unet_path = "stabilityai/stable-diffusion-xl-base-1.0"
# pretrained_image_encoder_path = "laion/CLIP-ViT-bigG-14-laion2B-39B-b160k"

In [ ]:
with open(unet_config_path) as f:
    unet_config = json.load(f)

In [ ]:
unet_config

In [ ]:
pretrained_unet = UNet2DConditionModel.from_pretrained(
    pretrained_unet_path, subfolder="unet"
)

In [ ]:
unet = UNet2DDobuleConditionModel.from_config(pretrained_unet.config)

In [ ]:
unet

In [ ]:
unet_parameters = unet.state_dict()
pretrained_unet_parameters = pretrained_unet.state_dict()
for key in unet_parameters:
    if key in pretrained_unet_parameters:
        if unet_parameters[key].shape == pretrained_unet_parameters[key].shape:
            unet_parameters[key] = pretrained_unet_parameters[key]
        elif unet_parameters[key].shape < pretrained_unet_parameters[key].shape:
            print(key)
            unet_parameters[key] = pretrained_unet_parameters[key][
                0 : unet_parameters[key].shape[0],
                0 : unet_parameters[key].shape[1],
            ]
        else:
            print(key)
            unet_parameters[key] = torch.nn.functional.pad(
                pretrained_unet_parameters[key],
                (
                    0,
                    unet_parameters[key].shape[1]
                    - pretrained_unet_parameters[key].shape[1],
                    0,
                    0,
                ),
            )

In [ ]:
unet.load_state_dict(unet_parameters)

In [ ]:
for key in unet.state_dict():
    if key in pretrained_unet.state_dict():
        print(unet.state_dict()[key] == pretrained_unet.state_dict()[key])
    else:
        print(key)

In [ ]:
# vae = diffusers.AutoencoderKL.from_pretrained("stabilityai/sdxl-vae")
vae = diffusers.AutoencoderKL.from_pretrained(
    "stabilityai/stable-diffusion-2-1", subfolder="vae"
)

In [ ]:
image_encoder = CLIPVisionModel.from_pretrained(pretrained_image_encoder_path)

In [ ]:
clip_image_processor = CLIPImageProcessor.from_pretrained(pretrained_image_encoder_path)

In [ ]:
scheduler = DDPMScheduler.from_pretrained(pretrained_unet_path, subfolder="scheduler")

In [ ]:
pipe = StableDiffusionReferenceOnlyPipeline(
    vae, image_encoder, clip_image_processor, unet, scheduler
)

In [ ]:
pipe.save_pretrained(
    "/home/aihao/workspace/DeepLearningContent/models/sd_reference_only/init_0.1"
)